In [1]:
!pip install httpimport

  Using cached httpimport-1.4.1-py2.py3-none-any.whl.metadata (11 kB)
Using cached httpimport-1.4.1-py2.py3-none-any.whl (17 kB)


In [2]:
import pandas as pd 

import boto3 
import sagemaker 
import httpimport 
import io 

from pyathena import connect 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
# initialize sagemaker sesssion and env var
sess = sagemaker.Session() 
bucket = sess.default_bucket() 
role = sagemaker.get_execution_role() 
region = boto3.Session().region_name
sm = boto3.client("sagemaker", region_name=region) 

In [4]:
# set s3 variables
# s3 = boto3.client('s3')
# bucket_name = 'jc-bucket-ads508-2025-finalproject'

# file_benefits = './employee_benefits_2010_2024.csv'
# s3_key_benefits = 'datasets/employee_benefits_2010_2024.csv'
s3_public_path = "s3://usd-team1-ads508/ld_data/"
s3_private_path = f"s3://{bucket}/ld_data"

print(f"Public path: {s3_public_path}")

Public path: s3://usd-team1-ads508/ld_data/


In [5]:
# import jcds 
with httpimport.github_repo('junclemente', 'jcds', ref='0.1.0'):
    import jcds.eda as jeda

In [6]:
# load dataset from s3 into dataframe
# obj = s3.get_object(Bucket=bucket_name, Key=s3_key_benefits)
# # read Excel file into dataframe
# df = pd.read_csv(io.BytesIO(obj['Body'].read()))

# print(df.head())

In [7]:
# store paths
%store s3_public_path
%store s3_private_path

Stored 's3_public_path' (str)
Stored 's3_private_path' (str)


In [8]:
# copy csv files from public to private bucket
!aws s3 cp --recursive {s3_public_path} {s3_private_path}/ --include "*.csv" > /dev/null 2>&1 

In [10]:
# list objects in bucket
s3 = boto3.client("s3") 
bucket_name = "usd-team1-ads508"
response = s3.list_objects_v2(Bucket=bucket_name)

# extract file details
if "Contents" in response:
    file_list = [{"Key": obj["Key"], "Size (KB)": obj["Size"] / 1024, "Last Modified": obj["LastModified"]} for obj in response["Contents"]]

    # convert to pandas df
    df_files = pd.DataFrame(file_list) 

    # display 
    print(df_files) 
else: 
    print("No files founds in the S3 bucket.")

                                   Key     Size (KB)             Last Modified
0        ab_data/analytic_data2019.csv   8042.529297 2025-03-20 23:33:07+00:00
1      ab_data/analytic_data2020_0.csv  11877.680664 2025-03-20 23:33:09+00:00
2        ab_data/analytic_data2021.csv  12185.751953 2025-03-20 23:33:10+00:00
3        ab_data/analytic_data2022.csv  12461.061523 2025-03-20 23:33:11+00:00
4      ab_data/analytic_data2023_0.csv  12232.161133 2025-03-20 23:33:13+00:00
5   jc_data/employee_benefits_2019.csv  22047.868164 2025-03-20 23:33:14+00:00
6   jc_data/employee_benefits_2020.csv  18644.913086 2025-03-20 23:33:15+00:00
7   jc_data/employee_benefits_2021.csv  21138.279297 2025-03-20 23:33:17+00:00
8   jc_data/employee_benefits_2022.csv  20985.699219 2025-03-20 23:33:18+00:00
9   jc_data/employee_benefits_2023.csv  21521.333008 2025-03-20 23:33:20+00:00
10                 ld_data/adult19.csv  26104.992188 2025-03-20 23:33:21+00:00
11                 ld_data/adult20.csv  29708.664062